In [1]:
import sys
sys.path.append("/Users/hirototakaura/opt/anaconda3/lib/python3.8/site-packages")
from osgeo import gdal, ogr, osr
from pyproj import Transformer
import pandas as pd
import numpy as np
import random
import gc
from geopy.distance import geodesic
from math import *
from makeBuffer import makeBuffer
import pickle

In [2]:
def cal_distance(pair1, pair2):
    dis = geodesic(pair1, pair2).m
    return dis


def cal_edgec(yx0, yx1, yx2, yx3):
    '''
    boxの各頂点を引数にとる　#[lat, long]
    各辺の中点を計算する
    2本の短辺の中点を返す #[lat, long], [lat, long]
    '''
    l0 = cal_distance(yx0, yx1)
    l1 = cal_distance(yx1, yx2)
    l2 = cal_distance(yx2, yx3)
    l3 = cal_distance(yx3, yx0)
    if l0 >= l1: #短辺がl1
        c1 = (np.array(yx1) + np.array(yx2)) / 2
        c2 = (np.array(yx3) + np.array(yx0)) / 2
        return c1, c2
    else:
        c1 = (np.array(yx0) + np.array(yx1)) / 2
        c2 = (np.array(yx2) + np.array(yx3)) / 2
        return c1, c2
    
    
def inpolygon(sx, sy, x, y):
    ''' 
    x[:], y[:]: polygon
    sx, sy: point
    '''     
    np = len(x)
    inside = False
    for i1 in range(np): 
        i2 = (i1+1)%np
        if min(x[i1], x[i2]) < sx < max(x[i1], x[i2]):
            #a = (y[i2]-y[i1])/(x[i2]-x[i1])
            #b = y[i1] - a*x[i1]
            #dy = a*sx+b - sy
            #if dy >= 0:
            if (y[i1] + (y[i2]-y[i1])/(x[i2]-x[i1])*(sx-x[i1]) - sy) > 0:
                inside = not inside

    return inside


def search_pair(df, id):
    '''
    一つの車両に注目した時にrearとfrontそれぞれにペアリングする車両のインデックスを返す
    '''
    
    #緯度で計算するか経度で計算するか
    dx = abs(df['fr_long'][id] - df['rear_long'][id])
    dy = abs(df['fr_lat'][id] - df['rear_lat'][id])
    if dx >= dy:
        #longの差で周囲の点との距離を計算
        #frontに注目
        if df['connect_fr'][id]:
            result_fr = None
        else:
            num = df['fr_long'][id]
            if df['fr_long'][id] - df['rear_long'][id] >= 0:
                #frontが右側
                fr_diff1 = df[df['fr_long'].sub(num) > 0]['fr_long'].sub(num).abs()
                rear_diff1 = df[df['rear_long'].sub(num) > 0]['rear_long'].sub(num).abs()
                if fr_diff1.empty or rear_diff1.empty:#注目している車両が端の場合リストは空になる可能性がある
                    result_fr = None
                else:
                    fr_id1 =  fr_diff1.idxmin()
                    rear_id1 =  rear_diff1.idxmin()
                    #距離が近い方のindexを採用
                    fr_fr = cal_distance([df['fr_lat'][id], df['fr_long'][id]], [df['fr_lat'][fr_id1], df['fr_long'][fr_id1]])
                    fr_rear = cal_distance([df['fr_lat'][id], df['fr_long'][id]], [df['rear_lat'][rear_id1], df['rear_long'][rear_id1]])
                    way = 'front' if fr_fr <= fr_rear else 'rear'
                    if way == 'front':
                        #front - front でペアリング
                        df.at[id, 'connect_fr'] = True
                        df.at[id, 'fr_connectTo'] = fr_id1
                        df.at[fr_id1, 'connect_fr'] = True
                        df.at[fr_id1, 'fr_connectTo'] = id
                        result_fr = fr_id1
                    else:
                        #front - rear　でペアリング
                        df.at[id, 'connect_fr'] = True
                        df.at[id, 'fr_connectTo'] = fr_id1
                        df.at[rear_id1, 'connect_rear'] = True
                        df.at[rear_id1, 'rear_connectTo'] = id
                        result_fr = rear_id1

            else:
                #frontが左側
                fr_diff1 = df[df['fr_long'].sub(num) < 0]['fr_long'].sub(num).abs()
                rear_diff1 = df[df['rear_long'].sub(num) < 0]['rear_long'].sub(num).abs()
                if fr_diff1.empty or rear_diff1.empty:
                    result_fr = None
                else:
                    fr_id1 =  fr_diff1.idxmin()
                    rear_id1 =  rear_diff1.idxmin()
                    #距離が近い方のindexを採用
                    fr_fr = cal_distance([df['fr_lat'][id], df['fr_long'][id]], [df['fr_lat'][fr_id1], df['fr_long'][fr_id1]])
                    fr_rear = cal_distance([df['fr_lat'][id], df['fr_long'][id]], [df['rear_lat'][rear_id1], df['rear_long'][rear_id1]])
                    way = 'front' if fr_fr <= fr_rear else 'rear'
                    if way == 'front':
                        #front - front でペアリング
                        df.at[id, 'connect_fr'] = True
                        df.at[fr_id1, 'fr_connectTo'] = fr_id1
                        df.at[fr_id1, 'connect_fr'] = True
                        df.at[fr_id1, 'fr_connectTo'] = id
                        result_fr = fr_id1
                    else:
                        #front - rear　でペアリング
                        df.at[id, 'connect_fr'] = True
                        df.at[id, 'fr_connectTo'] = fr_id1
                        df.at[rear_id1, 'connect_rear'] = True
                        df.at[rear_id1, 'rear_connectTo'] = id
                        result_fr = rear_id1
        
        #rearに注目
        if df['connect_rear'][id]:
            result_rear = None
        else:
            num = df['rear_long'][id]
            if df['rear_long'][id] - df['fr_long'][id] >= 0:
                #rearが右側
                fr_diff2 = df[df['fr_long'].sub(num) > 0]['fr_long'].sub(num).abs()
                rear_diff2 = df[df['rear_long'].sub(num) > 0]['rear_long'].sub(num).abs()
                if fr_diff2.empty or rear_diff2.empty:#注目している車両が端の場合リストは空になる可能性がある
                    result_rear = None
                else:
                    fr_id2 =  fr_diff2.idxmin()
                    rear_id2 =  rear_diff2.idxmin()
                    #距離が近い方のindexを採用
                    rear_fr = cal_distance([df['rear_lat'][id], df['rear_long'][id]], [df['fr_lat'][fr_id2], df['fr_long'][fr_id2]])
                    rear_rear = cal_distance([df['rear_lat'][id], df['rear_long'][id]], [df['rear_lat'][rear_id2], df['rear_long'][rear_id2]])
                    way = 'front' if rear_fr <= rear_rear else 'rear'
                    if way == 'front':
                        #rear - front でペアリング
                        df.at[id, 'connect_rear'] = True
                        df.at[id, 'rear_connectTo'] = fr_id2
                        df.at[fr_id2, 'connect_fr'] = True
                        df.at[fr_id2, 'fr_connectTo'] = id
                        result_rear = fr_id2
                    else:
                        #rear - rear　でペアリング
                        df.at[id, 'connect_rear'] = True
                        df.at[id, 'rear_connectTo'] = fr_id2
                        df.at[rear_id2, 'connect_rear'] = True
                        df.at[rear_id2, 'rear_connectTo'] = id
                        result_rear = rear_id2

            else:
                #rearが左側
                fr_diff2 = df[df['fr_long'].sub(num) < 0]['fr_long'].sub(num).abs()
                rear_diff2 = df[df['rear_long'].sub(num) < 0]['rear_long'].sub(num).abs()
                if fr_diff2.empty or rear_diff2.empty: 
                    result_rear = None
                else:
                    fr_id2 =  fr_diff2.idxmin()
                    rear_id2 =  rear_diff2.idxmin()
                    #距離が近い方のindexを採用
                    rear_fr = cal_distance([df['rear_lat'][id], df['rear_long'][id]], [df['fr_lat'][fr_id2], df['fr_long'][fr_id2]])
                    rear_rear = cal_distance([df['rear_lat'][id], df['rear_long'][id]], [df['rear_lat'][rear_id2], df['rear_long'][rear_id2]])
                    way = 'front' if rear_fr <= rear_rear else 'rear'
                    if way == 'front':
                        #rear - front でペアリング
                        df.at[id, 'connect_rear'] = True
                        df.at[id, 'rear_connectTo'] = fr_id2
                        df.at[fr_id2, 'connect_fr'] = True
                        df.at[fr_id2, 'fr_connectTo'] = id
                        result_rear = fr_id2
                    else:
                        #rear - rear　でペアリング
                        df.at[id, 'connect_rear'] = True
                        df.at[id, 'rear_connectTo'] = fr_id2
                        df.at[rear_id2, 'connect_rear'] = True
                        df.at[rear_id2, 'rear_connectTo'] = id
                        result_rear = rear_id2
        
        return [result_fr, result_rear] 
    
    else:
        #latの差で周囲の点との距離を計算
        #frontに注目
        if df['connect_fr'][id]:
            result_fr = None
        else:
            num = df['fr_lat'][id]
            if df['fr_lat'][id] - df['rear_long'][id] >= 0:
                #frontが右側
                fr_diff1 = df[df['fr_lat'].sub(num) > 0]['fr_lat'].sub(num).abs()
                rear_diff1 = df[df['rear_lat'].sub(num) > 0]['rear_lat'].sub(num).abs()
                if fr_diff1.empty or rear_diff1.empty:#注目している車両が端の場合リストは空になる可能性がある
                    result_fr = None
                else:
                    fr_id1 =  fr_diff1.idxmin()
                    rear_id1 =  rear_diff1.idxmin()
                    #距離が近い方のindexを採用
                    fr_fr = cal_distance([df['fr_lat'][id], df['fr_long'][id]], [df['fr_lat'][fr_id1], df['fr_long'][fr_id1]])
                    fr_rear = cal_distance([df['fr_lat'][id], df['fr_long'][id]], [df['rear_lat'][rear_id1], df['rear_long'][rear_id1]])
                    way = 'front' if fr_fr <= fr_rear else 'rear'
                    if way == 'front':
                        #front - front でペアリング
                        df.at[id, 'connect_fr'] = True
                        df.at[id, 'fr_connectTo'] = fr_id1
                        df.at[fr_id1, 'connect_fr'] = True
                        df.at[fr_id1, 'fr_connectTo'] = id
                        result_fr = fr_id1
                    else:
                        #front - rear　でペアリング
                        df.at[id, 'connect_fr'] = True
                        df.at[id, 'fr_connectTo'] = fr_id1
                        df.at[rear_id1, 'connect_rear'] = True
                        df.at[rear_id1, 'rear_connectTo'] = id
                        result_fr = rear_id1

            else:
                #frontが左側
                fr_diff1 = df[df['fr_lat'].sub(num) < 0]['fr_lat'].sub(num).abs()
                rear_diff1 = df[df['rear_lat'].sub(num) < 0]['rear_lat'].sub(num).abs()
                if fr_diff1.empty or rear_diff1.empty: 
                    result_fr = None
                else:
                    fr_id1 =  fr_diff1.idxmin()
                    rear_id1 =  rear_diff1.idxmin()
                    #距離が近い方のindexを採用
                    fr_fr = cal_distance([df['fr_lat'][id], df['fr_long'][id]], [df['fr_lat'][fr_id1], df['fr_long'][fr_id1]])
                    fr_rear = cal_distance([df['fr_lat'][id], df['fr_long'][id]], [df['rear_lat'][rear_id1], df['rear_long'][rear_id1]])
                    way = 'front' if fr_fr <= fr_rear else 'rear'
                    if way == 'front':
                        #front - front でペアリング
                        df.at[id, 'connect_fr'] = True
                        df.at[id, 'fr_connectTo'] = fr_id1
                        df.at[fr_id1, 'connect_fr'] = True
                        df.at[fr_id1, 'fr_connectTo'] = id
                        result_fr = fr_id1
                    else:
                        #front - rear　でペアリング
                        df.at[id, 'connect_fr'] = True
                        df.at[id, 'fr_connectTo'] = fr_id1
                        df.at[rear_id1, 'connect_rear'] = True
                        df.at[rear_id1, 'rear_connectTo'] = id
                        result_fr = rear_id1
                        
        #rearに注目
        if df['connect_rear'][id]:
            result_rear = None
        else:
            num = df['rear_lat'][id]
            if df['rear_lat'][id] - df['fr_lat'][id] >= 0:
                #rearが右側
                fr_diff2 = df[df['fr_lat'].sub(num) > 0]['fr_lat'].sub(num).abs()
                rear_diff2 = df[df['rear_lat'].sub(num) > 0]['rear_lat'].sub(num).abs()
                if fr_diff2.empty or rear_diff2.empty:#注目している車両が端の場合リストは空になる可能性がある
                    result_rear = None
                else:
                    fr_id2 =  fr_diff2.idxmin()
                    rear_id2 =  rear_diff2.idxmin()
                    #距離が近い方のindexを採用
                    rear_fr = cal_distance([df['rear_lat'][id], df['rear_long'][id]], [df['fr_lat'][fr_id2], df['fr_long'][fr_id2]])
                    rear_rear = cal_distance([df['rear_lat'][id], df['rear_long'][id]], [df['rear_lat'][rear_id2], df['rear_long'][rear_id2]])
                    way = 'front' if rear_fr <= rear_rear else 'rear'
                    if way == 'front':
                        #rear - front でペアリング
                        df.at[id, 'connect_rear'] = True
                        df.at[id, 'rear_connectTo'] = fr_id2
                        df.at[fr_id2, 'connect_fr'] = True
                        df.at[fr_id2, 'fr_connectTo'] = id
                        result_rear = fr_id2
                    else:
                        #rear - rear　でペアリング
                        df.at[id, 'connect_rear'] = True
                        df.at[id, 'rear_connectTo'] = fr_id2
                        df.at[rear_id2, 'connect_rear'] = True
                        df.at[rear_id2, 'rear_connectTo'] = id
                        result_rear = rear_id2

            else:
                #rearが左側
                fr_diff2 = df[df['fr_lat'].sub(num) < 0]['fr_lat'].sub(num).abs()
                rear_diff2 = df[df['rear_lat'].sub(num) < 0]['rear_lat'].sub(num).abs()
                if fr_diff2.empty or rear_diff2.empty: 
                    result_rear = None
                else:
                    fr_id2 =  fr_diff2.idxmin()
                    rear_id2 =  rear_diff2.idxmin()
                    #距離が近い方のindexを採用
                    rear_fr = cal_distance([df['rear_lat'][id], df['rear_long'][id]], [df['fr_lat'][fr_id2], df['fr_long'][fr_id2]])
                    rear_rear = cal_distance([df['rear_lat'][id], df['rear_long'][id]], [df['rear_lat'][rear_id2], df['rear_long'][rear_id2]])
                    way = 'front' if rear_fr <= rear_rear else 'rear'
                    if way == 'front':
                        #rear - front でペアリング
                        df.at[id, 'connect_rear'] = True
                        df.at[id, 'rear_connectTo'] = fr_id2
                        df.at[fr_id2, 'connect_fr'] = True
                        df.at[fr_id2, 'fr_connectTo'] = id
                        result_rear = fr_id2
                    else:
                        #rear - rear　でペアリング
                        df.at[id, 'connect_rear'] = True
                        df.at[id, 'rear_connectTo'] = fr_id2
                        df.at[rear_id2, 'connect_rear'] = True
                        df.at[rear_id2, 'rear_connectTo'] = id
                        result_rear = rear_id2
                        
        return [result_fr, result_rear]
    
    
#ペアリングした点を探索
def makeDistance(df, id):
    fr = df['fr_connectTo'][id]
    way = 'rear' if df['rear_connectTo'][fr] == id else 'front'
    if way == 'rear':
        long = [df['fr_long'][id], df['rear_long'][fr]]
        lat = [df['fr_lat'][id], df['rear_lat'][fr]]
    else:
        long = [df['fr_long'][id], df['fr_long'][fr]]
        lat = [df['fr_lat'][id], df['fr_lat'][fr]]
    line = ogr.Geometry(ogr.wkbLineString)
    for x, y in zip(long, lat):
        line.AddPoint(x, y)
        
    return line, fr
#2点からラインを作成
def makeline(pt1, pt2):
    line = ogr.Geometry(ogr.wkbLineString)
    line.AddPoint(pt1[0], pt1[1])
    line.AddPoint(pt2[0], pt2[1])
    return line


def transcood(pts, n):
    transformer = Transformer.from_crs("epsg:4326", f"epsg:{n+6668}")
    pt_new = []
    for pt in transformer.itransform(pts):
        pt_new.append(pt)
    return pt_new
        
'''
点から線分に垂線を下すことができるかどうか調べる
線分の端点はpt1とpt2
座標は(lat, long)
点はtarget
'''
def isDrawvertline(pt1, pt2, target):
    l1 = geodesic(target, pt1).m
    l2 = geodesic(target, pt2).m
    if l1 > l2:
        #始点をpt1に
        temp = pt1
        pt1 = pt2
        pt2 = temp    
        
    a = np.array(pt1)
    b = np.array(pt2)
    c = np.array(target)
    x = c - a
    y = b - a
    dp = np.dot(x, y)
    x_norm = np.linalg.norm(x)
    y_norm = np.linalg.norm(y)
    cos = dp / (x_norm * y_norm  + 1e-10)
    return cos

'''
cosを計算する
isDrawvertlineと違うのはベクトルの原点を確定するところ
'''
def cos(pt1, pt2, pt3):
    #pt1を原点とする
    a = np.array(pt1)
    b = np.array(pt2)
    c = np.array(pt3)
    x = c - a
    y = b - a
    dp = np.dot(x, y)
    x_norm = np.linalg.norm(x)
    y_norm = np.linalg.norm(y)
    cos = dp / (x_norm * y_norm  + 1e-10)
    return cos

def calvertline(pt1, pt2, target):
    pt1 = transcood([pt1], 6)[0]
    pt2 = transcood([pt2], 6)[0]
    target = transcood([target], 6)[0]
    
    c = np.array(target)
    a = np.array(pt1)
    b = np.array(pt2)
    x = c - a
    y = b - a
    x_norm = np.linalg.norm(x)
    y_norm = np.linalg.norm(y)
    L = np.cross(x, y) / (y_norm) #外積から二つのベクトルのなす角のsinをもとめる
    if L < 0:
        L = -L
    
    return L


def LenLine(line):
    lengths = []
    pt1 = line.GetPoint(0)
    for count in range(1, line.GetPointCount()):
        pt2 = line.GetPoint(count)
        lengths.append(geodesic([pt1[1], pt1[0]], [pt2[1], pt2[0]]).m)
        pt1 = pt2

    return sum(lengths)


'''
pointsには(lat, longの点が格納されている)
'''
def LenLinePoints(points):
    lengths = []
    pt1 = points[0]
    for count in range(1, len(points)):
        pt2 = points[count]
        lengths.append(geodesic(pt1, pt2).m)
        pt1 = pt2

    return sum(lengths)
    

'''
2点それぞれから車線に下ろした垂線とその車線の交点で切り取られた車線の線分の距離を求める
fr: (lat, long)
rear: (lat, long)
line: LINESTRING
'''
def DisFromRdCL(fr, rear, line):
    #frとrearから下ろした垂線とlineの交点を求める
    ends = []
    #垂線と車線の交点二つ求める
    for target in [fr, rear]:
        pt1 = line.GetPoint(0)
        pt1 = (pt1[1], pt1[0])
        for count in range(1, line.GetPointCount()):
            pt2 = line.GetPoint(count)
            pt2 = (pt2[1], pt2[0])
            cos = isDrawvertline(pt1, pt2, target)
            if cos >= 0:
                l1 = geodesic(target, pt1).m
                l2 = geodesic(target, pt2).m
                #始点をpt1に
                pt_a = pt1
                pt_b = pt2
                if l1 > l2:
                    #targetがpt２に近いなら始点をpt2に
                    pt_a = pt2
                    pt_b = pt1
                
                c = np.array(target)
                a = np.array(pt_a)
                b = np.array(pt_b)
                
                k = c - a
                l = b - a
                k_norm = np.linalg.norm(k)
                l_norm = np.linalg.norm(l)
                
                v = pt_a + ((k_norm * cos) / l_norm) * l #垂線と車線の交点
                
                ends.append(v)
                break
            pt1 = pt2
    #frとrearをから垂線を下ろせない状況では2点をつないで返す
    if len(ends) < 2:
        print('False')
        length = geodesic(fr, rear).m
        line_dis = ogr.Geometry(ogr.wkbLineString)
        line_dis.AddPoint(fr[1], fr[0])
        line_dis.AddPoint(rear[1], rear[0])
        return length, line_dis
     #2つの端点間にあるlineの点をpointsに加える   
    #endsは(lat, long)
    pt1 = ends[0]
    pt2 = ends[1]
    points = []
    line_dis = ogr.Geometry(ogr.wkbLineString) #二つの端点とその間にあるlineの点から新しいline_disを作成する
    #line_dis.AddPoint(pt1[1], pt1[0])
    for count in range(line.GetPointCount()):
        target = line.GetPoint(count)
        target = (target[1], target[0])
        if isDrawvertline(pt1, pt2, target) >= 0:
            points.append(target)
    #端点を近い方に挿入    
    if points:
        pt_near = nearpt(pt1, pt2, points[-1]) #ラインの終点に近い点はfrとrearのどっち？
        if not (pt_near -np.array(pt1)).all():
            points.append(pt1)
            points.insert(0, pt2)
        else:
            points.append(pt2)
            points.insert(0, pt1)
    
    #pt1とpt2とpointsを繋いだ時の距離を計算
        for p in points:
            line_dis.AddPoint(p[1], p[0])
        length = LenLinePoints(points)

    else:
        line_dis.AddPoint(pt1[1], pt1[0])
        line_dis.AddPoint(pt2[1], pt2[0])
        length = LenLinePoints(ends)
        
    return length, line_dis


#走行速度[m/s]
def calSpeed(distanse):
    d = distanse
    tau = 0.75
    mu = 0.7
    g = 9.8
    v = np.sqrt((tau * tau) * (mu * mu) * (g * g) + 2 * d * mu * g) - tau * mu * g
    
    return v 


#断面交通量
'''
L[km]
Nperiod[台]

'''
def calNt(L, Nperiod, V):
    if Nperiod != 0:
        rho = Nperiod / L
        Nt = rho * (V)
    else:
        Nt = 0
    return Nt
#A特性音響パワーレベル
def calLwa(v, vehicleclass):
    if vehicleclass == 'small':
        a = 45.8
    else:
        a = 53.2
        
    C = 0
    b = 30
    Lwa = a + b * np.log10(v) + C
    
    return Lwa

'''
並列している２つ以上のboxうち中心線に最も近いもの以外を削除する
df：　boxのDataframe
line： 中心線のLINESTRING
'''
def remLineup(df, line):
    rem = []
    for id1 in df.index.to_numpy():
        if id1 in rem:
            continue
        pt1 = (df['fr_lat'][id1], df['fr_long'][id1])
        pt2 = (df['rear_lat'][id1], df['rear_long'][id1])
        for id2 in df.index.to_numpy():
            if id1 in rem or id2 in rem:
                break
            for direct in ['fr', 'rear']:
                if id1 in rem or id2 in rem:
                    break
                target = (df[f'{direct}_lat'][id2], df[f'{direct}_long'][id2])
                l1 = geodesic(target, pt1).m
                l2 = geodesic(target, pt2).m
                if l1 > l2:
                    #始点をpt1に
                    temp = pt1
                    pt1 = pt2
                    pt2 = temp
                if isDrawvertline(pt1, pt2, target) > 0:#並んでいるかどうか
                    #車線から遠い方を削除
                    vertL = [] #重心から中心線までの距離の初期化
                    for id3 in [id1, id2]:
                        target2 = (df['c_lat'][id3], df['c_long'][id3])
                        #中心線を構成する線分のうちtarget１と２から垂線を下ろすことができる線分を探索する
                        linept1 = line.GetPoint(0)
                        for count in range(1, line.GetPointCount()):
                            linept2 = line.GetPoint(count)
                            linept_a = (linept1[1], linept1[0])
                            linept_b = (linept2[1], linept2[0])
                            l3 = geodesic(target2, linept_a).m
                            l4 = geodesic(target2, linept_b).m
                            if l3 > l4:
                                #始点をlinept_aに
                                temp = linept_a
                                linept_a = linept_b
                                linept_b = temp
                            if isDrawvertline(linept_a, linept_b, target2) >= 0:
                                L = calvertline(linept_a, linept_b, target2)
                                #print('L: '.format(L))
                                vertL.append((id3, L))
                                break
                            linept1 = linept2
                    #print(vertL)
                    #中心線から遠いboxを削除
                    if len(vertL) < 2:
                        #垂線を2本引けない場合ランダムでどちらかを削除
                        delid = random.choice([id1, id2])
                        df = df.drop(delid)
                        #print('random drop {}'.format(delid))
                        rem.append(delid)
                    elif vertL[0][1] >= vertL[1][1]:
                        delid = vertL[0][0]
                        df = df.drop(delid)
                        rem.append(delid)
                        #print('drop {}'.format(delid))
                        break
                    elif vertL[0][1] >= vertL[1][1]:
                        delid = vertL[1][0]
                        df = df.drop(delid)
                        rem.append(delid)
                        #print('drop {}'.format(delid))

    return df


'''
lineの始点付近もしくは終点付近の車両を検索
このとき検索対象の車両はfront, rearのどちらかだけがペアリングしているものとする
mode：　始点付近か終点付近かを決める
'''
def SearchNearestCar(line, box, pt1, mode):
    #車両を検出
    filter = makeBuffer(line, 1)
    #insideを初期化
    box.loc[:, 'inside'] = False
    for col in box.index.to_numpy():
        box.at[col, 'inside'] = inpolygon(box['c_long'][col], box['c_lat'][col], filter['long'], filter['lat'])
    box_grp2 = box[box['inside'] == True]
    if box_grp2.empty:
        return -1
    
#     RdCL_work = RdCL_df.copy()
#     if mode == 'end':
#         pt1 = [RdCL_df['end_lat'][id], RdCL_df['end_long'][id]]
#     else:
#         pt1 = [RdCL_df['start_lat'][id], RdCL_df['start_long'][id]]
    #端点に近いペアリング済み車両を検索(frontだから前方、rearだから後方というわけではないことに注意)
    box_temp = box_grp2.copy()
    box_temp.loc[:, 'diff'] = (box_temp['c_long'] - pt1[1]).abs() + (box_temp['c_lat'] - pt1[0]).abs()
    nearcar_id = box_temp['diff'].idxmin()
    return nearcar_id


'''
ペアリングされた２つのboxの車間距離、車速を計算する
'''
#wktは一つしか持てないので前方と後方で
def Connect(nearcar_id, end_id, box):
#connectToは初期化されるので無意味
    pt1 = nearpt([box['fr_lat'][nearcar_id], box['fr_long'][nearcar_id]],\
                 [box['rear_lat'][nearcar_id], box['rear_long'][nearcar_id]], \
                 [box['fr_lat'][end_id], box['fr_long'][end_id]])
    pt2 = nearpt([box['fr_lat'][end_id], box['fr_long'][end_id]],\
                 [box['rear_lat'][end_id], box['rear_long'][end_id]], \
                 [box['rear_lat'][nearcar_id], box['rear_long'][nearcar_id]])
    
    line_dis = makeline([pt1[1], pt1[0]], [pt2[1], pt2[0]])
    dis = LenLine(line_dis)

    box.at[nearcar_id, 'distanse'].append(dis) #boxに追加する
    box.at[nearcar_id, 'v'].append(calSpeed(dis) * 3.6)
    box.at[nearcar_id, 'WKT_dis'] = line_dis.ExportToWkt()
    
    
'''
二つの点から近い方の点を返す
targetに車線の端点を入れてpt1、pt２に車両のfrとrearを入れ
近い方を先頭とするために使用
'''
def nearpt(pt1, pt2, target):
    l1 = geodesic(pt1, target)
    l2 = geodesic(pt2, target)
    if l1 >= l2:
        return pt2
    else:
        return pt1
    

'''
bboxのdfからバッファに入る車両を検索してペアリング、dfにペアリングの情報を書き込む
fr_connectToとrear_conncetToはDetectionAndPairing内でしか保存されない
'''
def DetectionAndPairing(box:pd.DataFrame, RdCL_df, car_list, line):
    #insideを初期化
    box.loc[:, 'inside'] = False
    #filter作成
    filter = makeBuffer(line, 1)
    #内外判定
    for col in box.index.to_numpy():
        box.at[col, 'inside'] = inpolygon(box['c_long'][col], box['c_lat'][col], filter['long'], filter['lat'])
        
    #buffer内の車両に対して処理を行う
    box_grp = box[box['inside'] == True]
    print(f'\tinside car: {box_grp.index.to_numpy()}')
    print('\tnum of cars is {} in id {} RdCL'.format(len(box_grp.index.to_list()), id))
    if box_grp.empty:
        return False
        
    #車両が並列している場合中心線から遠い車両をbox_grから削除
    box_grp = remLineup(box_grp, line)
    #boxのペアリング
    count = 0
    for col in box_grp.index.to_numpy():
        count+= 1
        car_list.append(col)
        num = search_pair(box_grp, col)
        front_id = num[0]
        rear_id = num[1]
        
        print(f'\tnow car: {col} front: {front_id}, rear: {rear_id}')
        
        if front_id == None and rear_id == None:
            continue
            
        elif front_id != None and rear_id == None:
            box.at[col, 'fr_connectTo'] = front_id
            #front_id側のboxもcolとつなげる
            l_fr = geodesic([box['fr_lat'][col], box['fr_long'][col]], [box['fr_lat'][front_id], box['fr_long'][front_id]])
            l_rear = geodesic([box['fr_lat'][col], box['fr_long'][col]], [box['rear_lat'][front_id], box['rear_long'][front_id]])
            if l_fr >= l_rear:
                box.at[front_id, 'rear_connectTo'] = col
            else:
                box.at[front_id, 'fr_connectTo'] = col
                
        elif front_id == None and rear_id != None:
            box.at[col, 'rear_connectTo'] = rear_id
            #rear_id側のboxもcolとつなげる
            l_fr = geodesic([box['rear_lat'][col], box['rear_long'][col]], [box['fr_lat'][rear_id], box['fr_long'][rear_id]])
            l_rear = geodesic([box['rear_lat'][col], box['rear_long'][col]], [box['rear_lat'][rear_id], box['rear_long'][rear_id]])
            if l_fr >= l_rear:
                box.at[rear_id, 'rear_connectTo'] = col
            else:
                box.at[rear_id, 'fr_connectTo'] = col
            
        else:
            box.at[col, 'fr_connectTo'] = front_id
            box.at[col, 'rear_connectTo'] = rear_id
            
            #front_id側のboxもcolとつなげる
            l_fr = geodesic([box['fr_lat'][col], box['fr_long'][col]], [box['fr_lat'][front_id], box['fr_long'][front_id]])
            l_rear = geodesic([box['fr_lat'][col], box['fr_long'][col]], [box['rear_lat'][front_id], box['rear_long'][front_id]])
            if l_fr >= l_rear:
                box.at[front_id, 'rear_connectTo'] = col
            else:
                box.at[front_id, 'fr_connectTo'] = col
            #rear_id側のboxもcolとつなげる
            l_fr = geodesic([box['rear_lat'][col], box['rear_long'][col]], [box['fr_lat'][rear_id], box['fr_long'][rear_id]])
            l_rear = geodesic([box['rear_lat'][col], box['rear_long'][col]], [box['rear_lat'][rear_id], box['rear_long'][rear_id]])
            if l_fr >= l_rear:
                box.at[rear_id, 'rear_connectTo'] = col
            else:
                box.at[rear_id, 'fr_connectTo'] = col
    #片方しかペアリングしていないboxのうちstartに近い方を起点とする
    box_end = box_grp[(box_grp['connect_fr'] == False) | (box_grp['connect_rear'] == False)]
#     print(box_end.loc[:, ['connect_fr', 'connect_rear']])
    st_long, st_lat, _ = line.GetPoint(0)
    box_end.loc[:, 'diff'] = (box_end['c_long'] - st_long).abs() + (box_end['c_lat'] - st_lat).abs()
    st_id = box_end['diff'].idxmin()
    
    idcn = st_id
    way = 'rear' if box_grp['connect_fr'][idcn] == False \
                else 'front'
    if way == 'rear':
        idnext = box['rear_connectTo'][idcn]#起点のbboxにペアリングしているbboxのid
        pt1 = [box['rear_lat'][idcn], box['rear_long'][idcn]]

    else:
        idnext = box_grp['fr_connectTo'][idcn]
        pt1 = [box_grp['fr_lat'][idcn], box_grp['fr_long'][idcn]]
        
    print(f'\tidnext: {idnext}')
    #ペアリングしているboxを始点側から順に見ていって距離、車速計算
    id_list = []
    while(True):
        print(f'\tidnext: {idnext}')
        if idnext is None:
            break
        if not idnext in box_grp.index.to_list():
            #交差点のバッファに入る車が既に車線のバッファでペアリングしているケース
            break
        id_list.append(idcn)
        if box['fr_connectTo'][idnext] == idcn:
            #frontに接続
            pt2 = [box['fr_lat'][idnext], box['fr_long'][idnext]]
            dis, line_dis = DisFromRdCL(pt1, pt2, line)
            box.at[idcn, 'distanse'].append(dis) #boxに追加する
            box.at[idcn, 'WKT_dis'] = line_dis.ExportToWkt()
            box.at[idcn, 'v'].append(calSpeed(dis) * 3.6)
            
            pt1 = [box['rear_lat'][idnext], box['rear_long'][idnext]]
            idcn = idnext
            idnext = box['rear_connectTo'][idnext]
        else:
            #rearに接続
            pt2 = [box['rear_lat'][idnext], box['rear_long'][idnext]]
            dis, line_dis= DisFromRdCL(pt1, pt2, line)
            box.at[idcn, 'distanse'].append(dis) #boxに追加する
            box.at[idcn, 'WKT_dis'] = line_dis.ExportToWkt()
            box.at[idcn, 'v'].append(calSpeed(dis) * 3.6)
            
            pt1 = [box['fr_lat'][idnext], box['fr_long'][idnext]]
            idcn = idnext
            idnext = box['fr_connectTo'][idnext]
    box['inside'] = False
    del box_grp
    return True


#Union-Findを使って並列している車線のグループ化
def find(x, par):
    if par[x] == x:
        return x
    else:
        par[x] = find(par[x],par) #経路圧縮
        return par[x]
    
def same(x, y, par):
    return find(x, par) == find(y, par)

def unite(x,y, par):
    x = find(x, par)
    y = find(y, par)
    if x == y:
        return 0
    if rank[x] < rank[y]:
        par[x] = y
    else:
        par[y] = x
        if rank[x] == rank[y]: rank[x] += 1


In [4]:
#boxを読み込む
box = pd.read_csv(box_path, encoding = "shift_jis")
box['WKT'] = box['WKT'].map(lambda x: ogr.CreateGeometryFromWkt(x))
#角頂点を取り出す
vertix = {'long0':[], 'lat0':[], 'long1':[], 'lat1':[], 'long2':[], 'lat2':[], 'long3':[], 'lat3':[]}
for col in box.index.to_numpy():
    line = box['WKT'][col]
    for count in range(line.GetPointCount() - 1): #各ラインは最初と最後の座標が同じだから　 - 1
        pair = line.GetPoint(count)
        vertix['long{}'.format(count)].append(pair[0])
        vertix['lat{}'.format(count)].append(pair[1])
vertix_df = pd.DataFrame(vertix)
box = box.join(vertix_df)      
#boxの情報をまとめる
'''
'c_long': bboxの中心の経度
'c_lat': bboxの中心の緯度
'fr_long':　bboxの前方(と検出されている)の経度
'fr_lat':　bboxの前方の緯度
'rear_long':　bboxの後方の経度
'rear_lat':　bboxの後方の緯度
'connect_fr':　bboxの前方にどの車両があるか確定しているかどうか
'connect_rear':　bboxの後方にどの車両があるか確定しているかどうか
'fr_connectTo':　bboxの前方にある車両のdf上のid
'rear_connectTo':　bboxの後方にある車両のdf上のid
'inside':　bboxがバッファに含まれているかどうか
'distanse':　bboxの車間距離
'WKT_dis':　bboxの車間距離のWKT
'v': bboxの速度
'class':　bboxの車種

'''
boxfeature = {'c_long':[], 'c_lat':[], 'fr_long':[],'fr_lat':[], 'rear_long':[], 'rear_lat':[],
              'connect_fr':[], 'connect_rear':[], 'fr_connectTo':[], 'rear_connectTo':[], 'inside':[],'distanse':[], 'WKT_dis':[],   'v':[], 'class': []}
#boxの中心点、短辺の中点を計算
for col in range(box.shape[0]):
    center_long = (box['long0'][col] + box['long1'][col] + box['long2'][col] + box['long3'][col]) / 4 
    center_lat = (box['lat0'][col] + box['lat1'][col] + box['lat2'][col] + box['lat3'][col]) / 4
    fr_c, rear_c = cal_edgec([box['lat0'][col], box['long0'][col]], [box['lat1'][col], box['long1'][col]], 
                       [box['lat2'][col], box['long2'][col]], [box['lat3'][col], box['long3'][col]])
    boxfeature['fr_long'].append(fr_c[1])
    boxfeature['fr_lat'].append(fr_c[0])
    boxfeature['rear_long'].append(rear_c[1])
    boxfeature['rear_lat'].append(rear_c[0])
    boxfeature['c_long'].append(center_long)
    boxfeature['c_lat'].append(center_lat)
    #後ろ(=rear)と前(=fr)が連結しているかのフラグを設定
    boxfeature['connect_fr'] = False
    boxfeature['connect_rear'] = False
    boxfeature['fr_connectTo'] = None
    boxfeature['rear_connectTo'] = None
    boxfeature['inside'] = False#フィルターに入っているかのフラグ
    boxfeature['distanse'].append([0, 0])
    boxfeature['WKT_dis'] = None
    boxfeature['v'].append([0, 0])
    boxfeature['class'] = None

boxfeature_df = pd.DataFrame(boxfeature)
box = box.join(boxfeature_df)
for id in box.index.to_numpy():
    box['distanse'][id].clear()
    box['v'][id].clear()


#車線データ読み込み
RdCL_df = pd.read_csv(RdCL_path)
RdCL_df.dropna(subset=['WKT'], inplace = True)
RdCL_df = RdCL_df.reset_index(drop = True)
RdCL_df['WKT'] = RdCL_df['WKT'].map(lambda x: ogr.CreateGeometryFromWkt(x))
#車線から車の進行方向の決定と車間距離の作成
anchor = {'start_long': [], 'start_lat': [], 'end_long':[], 'end_lat':[]}
for id in RdCL_df.index.to_numpy():
    line = RdCL_df['WKT'][id]
    st = line.GetPoint(0)
    end = line.GetPoint(line.GetPointCount() - 1)
    #車線の起点
    anchor['start_long'].append(st[0])
    anchor['start_lat'].append(st[1])
    #車線の終点
    anchor['end_long'].append(end[0])
    anchor['end_lat'].append(end[1])
RdCL_df = RdCL_df.join(pd.DataFrame(anchor))
#各車線の長さを計算
if cal_Rdlen:
    RdLen = {'Length':[]}
    for id in RdCL_df.index.to_numpy():
        line = RdCL_df['WKT'][id]
        lengths = []
        pt1 = line.GetPoint(0)
        for count in range(1, line.GetPointCount()):
            pt2 = line.GetPoint(count)
            pair1 = [pt1[1], pt1[0]]
            pair2 = [pt2[1], pt2[0]]
            length = geodesic(pair1, pair2).m
            lengths.append(length)
            pt1 = pt2
        RdLen['Length'].append(sum(lengths))  
    RdLen_df = pd.DataFrame(RdLen)
    RdCL_df = RdCL_df.join(RdLen_df)

    #区間長計算部分
    #車線グループから並列した車線の長い方を選ぶ
    idnum = len(RdCL_df.index.to_list())
    check = np.zeros((idnum, idnum)) #どれとどれが並列した車線かを管理するフラグ

    for id in RdCL_df.index.to_numpy():
        line = RdCL_df['WKT'][id]
        for count in range(line.GetPointCount()):
            pt = line.GetPoint(count)
            pt = [pt[1], pt[0]]
            for id2 in RdCL_df.index.to_numpy():
                #すでに並列していることがわかっているならスキップ
                if check[id][id2] == 1\
                    or id == id2:#同じ道路を参照したらスキップ
                    continue
                line2 = RdCL_df['WKT'][id2]
                end1 = line2.GetPoint(0)
                end1= [end1[1], end1[0]]
                for count2 in range(1, line2.GetPointCount()):#idの車線の点からid2の車線の点２つからの線分に垂線が引けるか確かめる
                    end2 = line2.GetPoint(count2)
                    end2= [end2[1], end2[0]]
                    len1 = geodesic(pt, end1).m
                    len2 = geodesic(pt, end2).m
                    if len1 > len2:
                        #始点をend1に
                        temp = end1
                        end1 = end2
                        end2 = temp
                    if isDrawvertline(end1, end2, pt) >= 0:
                        vertL = calvertline(end1, end2, pt)
                        if vertL <= lineup_th: #線上の点から別の道路に下ろした垂線がlineout_th以下なら隣り合っていることにする
                            print('id {} RdCL and id {} RDCL are lines up'.format(id, id2))
                            check[id][id2] = 1
                            check[id2][id] = 1
                            break
                    end1 = end2


    rank = [0] * len(RdCL_df.index.to_list())           
    Rdpar = [i for i in RdCL_df.index.to_list()]         
    for i in RdCL_df.index.to_numpy():
        for j in RdCL_df.index.to_numpy():
            if check[i][j] == 1:
                unite(i, j, Rdpar)
    #minl_dicに書くグループの車線の長さとidを追加して降順にソート            
    minl_dic = {}
    print(list(set(Rdpar)))
    for i in list(set(Rdpar)):
        minl_dic[str(i)] = []
    for i in range(len(Rdpar)):
        minl_dic[str(Rdpar[i])].append([RdCL_df['Length'][i], i])
    for i in minl_dic.keys():
        minl_dic[i].sort()
        minl_dic[i].reverse()
    #各グループの最長の車線のidと距離
    maxRd_len  = []
    maxRd_id = []
    for i in minl_dic.keys():
        maxRd_len.append(minl_dic[i][0][0])
        maxRd_id.append(minl_dic[i][0][1])

    with open(RdGrp_path, "wb") as tf:
        pickle.dump(minl_dic,tf)
else:
    #計算済みの場合はpickleファイルからRdparを復元
    with open(RdGrp_path, "rb") as tf:
        minl_dic = pickle.load(tf)
    
    Rdpar = [i for i in RdCL_df.index.to_list()]
    for key in minl_dic.keys():
        for l in minl_dic[key]:
            Rdpar[l[1]] = int(key)
    
    #各グループの最長の車線のidと距離
    maxRd_len  = []
    maxRd_id = []
    for i in minl_dic.keys():
        maxRd_len.append(minl_dic[i][0][0])
        maxRd_id.append(minl_dic[i][0][1])
    
    
#交差点などの車線がない部分をつないだ距離を計算
#車線の終点(=end)と別の車線の起点(=st2)を繋いでいく
addLen = []
for grp in list(set(Rdpar)):
    lens = []
    for id in RdCL_df.index.to_list():
        if not find(id, Rdpar) == grp:
            continue
        end = [RdCL_df['end_lat'][id], RdCL_df['end_long'][id]]
        st = [RdCL_df['start_lat'][id], RdCL_df['start_long'][id]]
        #endから一番近い始点との距離を追加
        for id2 in RdCL_df.index.to_list():
            if same(id, id2, Rdpar):#idとid2が同じグループならスキップ
                continue
            st2 = [RdCL_df['start_lat'][id2], RdCL_df['start_long'][id2]]
            l1 = geodesic(st, st2).m
            l2 = geodesic(end, st2).m
            if l1 > l2: #l１ < l2の時車線は繋がない
                lens.append(l2)
    if lens:
        addLen.append(min(lens))

#区間長さを計算
#各グループの最長車線長さ　＋　交差点
Lperiod = sum(maxRd_len) + sum(addLen)
print('the most long road lenghts of each group: {}\neach id: {}'.format(maxRd_len, maxRd_id))
print('lengths of intersections: {}'.format(addLen))


car_list = [] #bufferに入る車両のidを格納して台数を数える
#各車線ごとにフィルターを作成して範囲内の車両に対して車速を計算
print('\nDetection and Paring')
for id in RdCL_df.index.to_numpy():
    print('\n    id {}'.format(id))
    line = RdCL_df['WKT'][id]
    Iscar = DetectionAndPairing(box, RdCL_df, car_list, line)
    

#交差点間の車両検出
print('\nDetection and Paring in intersection')
for id in RdCL_df.index.to_numpy():
    print(f'id {id}')
    line = RdCL_df['WKT'][id]
    for mode in ['end', 'start']:
        flag_corner = False #コーナーケース(lineがグループの端の車線かどうかのフラグ)
        print(f'mode {mode}')
        if mode == 'end' :
            #終点に着目
            pt1 = [RdCL_df['end_lat'][id], RdCL_df['end_long'][id]]
            pt1_oth = [RdCL_df['start_lat'][id], RdCL_df['start_long'][id]]#pt1の反対側の点
            pt1_oth2 = line.GetPoint(line.GetPointCount() - 2)#pt1の一個前の点
            pt1_oth2 = [pt1_oth2[1], pt1_oth2[0]]
            nearcar_id = SearchNearestCar(line, box, pt1, 'end')
            if nearcar_id == -1:
                print(f'\tRd id{id} this area has no car')
                continue
        else:
            #始点に着目
            pt1 = [RdCL_df['start_lat'][id], RdCL_df['start_long'][id]]
            pt1_oth = [RdCL_df['end_lat'][id], RdCL_df['end_long'][id]]
            pt1_oth2 = line.GetPoint(1)#pt1の一個次の点
            pt1_oth2 = [pt1_oth2[1], pt1_oth2[0]]
            nearcar_id = SearchNearestCar(line, box, pt1, 'start')
            if nearcar_id == -1:
                print(f'\tRd id{id} this area has no car')
                continue

        if mode == 'end' :
            #車線の端点に一番近い車両の交差点までの距離(endの場合だけ考える)
            pt_endcar = nearpt([box['fr_lat'][nearcar_id], box['fr_long'][nearcar_id]],\
                               [box['rear_lat'][nearcar_id], box['rear_long'][nearcar_id]], pt1)
            disIS = geodesic(pt1, pt_endcar).m

        #現在の車線と近い車線(並列除く)の端点を検索
        #全ての端点との距離を計算して0でない最も近い一点を選ぶ
        ptlen = []
        if mode == 'end' :
            for id2 in RdCL_df.index.to_numpy():
                if same(id, id2, Rdpar):
                    continue
                pt2 = [RdCL_df['start_lat'][id2], RdCL_df['start_long'][id2]]
                cos_end = cos(pt1, pt2, pt1_oth2)
                if cos_end <= 0:
                    ptlen.append([geodesic(pt1, pt2).m, id2])
        else:
            for id2 in RdCL_df.index.to_numpy():
                if same(id, id2, Rdpar):
                    continue
                pt2 = [RdCL_df['end_lat'][id2], RdCL_df['end_long'][id2]]
                cos_end = cos(pt1, pt2, pt1_oth2)
                if cos_end <= 0:
                    ptlen.append([geodesic(pt1, pt2).m, id2])
        if len(ptlen) == 0:
            flag_corner = True
        else:
            flag_corner = False
            ptlen.sort(key=lambda x: x[0])
            minid = ptlen[0][1]
            #print(f'\tminid: {minid}')
        
        if mode == 'end':
            pt_min = [RdCL_df['start_lat'][minid], RdCL_df['start_long'][minid]]
        else:
            pt_min = [RdCL_df['end_lat'][minid], RdCL_df['end_long'][minid]]

        #進行方向に点をつなぐためのif
        if not flag_corner:
            if mode == 'end':
                lineIS = makeline([pt1[1], pt1[0]], [pt_min[1], pt_min[0]])
            else:
                lineIS = makeline([pt_min[1], pt_min[0]], [pt1[1], pt1[0]])  
            #交差点バッファ内の車両を検出〜ペアリング    
            filterIS = makeBuffer(lineIS, 1)
            Iscar = DetectionAndPairing(box, RdCL_df, car_list, lineIS)
        if mode == 'end' :
            if flag_corner:
                if disIS <= th_stop:
                    #赤信号とする先頭の車は速度0
                        box.at[nearcar_id, 'distanse'].append(0) #boxに追加する
                        box.at[nearcar_id, 'v'].append(0)
                        print('\tcase 2')
                else:
                    #交差点までを車間距離とする
                    dis, line_dis = DisFromRdCL(pt_endcar, pt1, line)
                    box.at[nearcar_id, 'distanse'].append(dis) #boxに追加する
                    box.at[nearcar_id, 'WKT_dis'] = line_dis.ExportToWkt()
                    box.at[nearcar_id, 'v'].append(calSpeed(dis) * 3.6)
                    print('\tcase 3')
            else:
                if Iscar: #交差点内に車両が存在
                    #filter内の最後尾と先頭を繋ぐ
                    end_id = SearchNearestCar(lineIS, box, pt1, 'start')
                    Connect(nearcar_id, end_id, box)
                    print('\tcase 1')
                else:
                    if disIS <= th_stop:
                        #赤信号とする先頭の車は速度0
                        box.at[nearcar_id, 'distanse'].append(0) #boxに追加する
                        box.at[nearcar_id, 'v'].append(0)
                        print('\tcase 2')
                    else:
                        #交差点までを車間距離とする
                        print('\tcase 3')
                        dis, line_dis = DisFromRdCL(pt_endcar, pt1, line)
                        box.at[nearcar_id, 'distanse'].append(dis) #boxに追加する
                        box.at[nearcar_id, 'WKT_dis'] = line_dis.ExportToWkt()
                        box.at[nearcar_id, 'v'].append(calSpeed(dis) * 3.6)
        else:
            if flag_corner:
                print('\tcase 6')
                pass
            else:
            #startに着目している時はバッファに車両があれば繋ぐだけ
                if Iscar: 
                    end_id = SearchNearestCar(lineIS, box, pt1, 'end')
                    if end_id != -1:
                        print(end_id, nearcar_id)
                        Connect(end_id, nearcar_id, box)
                        print('\tcase 4')
                else:
                    print('\tcase 5')
                    pass
        box['inside'] = False
    print('\n')

#バッファ内の車両台数 
car_list = list(set(car_list))
car_num = len(car_list)

print(car_num)
#速度,　車間距離をまとめる
V = {'Speed': [], 'Distanse': []}
for id in box.index.to_numpy():
    #速度は算術平均を代表値とする
    v_list = box['v'][id]
    v_sum = sum(v_list)
    d_list = box['distanse'][id]
    d_sum = sum(d_list)
    if len(v_list) != 0:
        V['Speed'].append(v_sum / len(v_list))
        V['Distanse'].append(d_sum / len(d_list))
    else:
        V['Speed'].append(0)
        V['Distanse'].append(0)
box = box.join(pd.DataFrame(V))

#車両のクラス分け
box.loc[box['L2'] < class_th, 'class'] = 'small'
box.loc[box['L2'] >= class_th, 'class'] = 'large'

box_work = box.loc[car_list, :].copy()

#断面交通量
if len(box_work[box_work['Speed'] != 0.00]) != 0:
    Vmean = sum(box_work['Speed']) / len(box_work[box_work['Speed'] != 0.00].index.to_list())#車速は全体の算術平均をとる
    box_work.loc[:, 'Nt'] = calNt(Lperiod / 1000, car_num, Vmean)
    num_small = len(box_work[box_work['class'] == 'small'].index.to_list())
    num_large = len(box_work[box_work['class'] == 'large'].index.to_list())
    id_small = box_work[box_work['class']=='small'].index.to_list()
    id_large = box_work[box_work['class']=='large'].index.to_list()

    box_work.loc[id_small, 'NtGrp'] = calNt(Lperiod / 1000, num_small, Vmean)
    box_work.loc[id_large, 'NtGrp'] = calNt(Lperiod / 1000, num_large, Vmean)
    #Lwa計算
    Lwa = {'Lwa':[]}
    for id in car_list:
        lwa = calLwa(Vmean, box['class'][id])
        Lwa['Lwa'].append(lwa)
    print(len(Lwa['Lwa']))
    box_work.loc[car_list, 'Lwa'] = box['class'].map(lambda x: calLwa(Vmean, x))

    box_work = box_work[box_work['WKT_dis'].notna()]
    #速度は全体の算術平均
    box_work.loc[:, 'Vmean'] = Vmean
    box_work.loc[:, 'Duration'] = (box_work['Distanse'] / box_work['Speed']) * 3.6

    box_out = box_work.loc[:, ['WKT_dis', 'Distanse', 'Speed','Vmean', 'Duration', 'class', 'Nt','NtGrp',  'Lwa']]
    #buffer内のbox書き出し
    box_out = box_out.rename(columns={'WKT_dis': 'WKT'})
    box_out.to_csv(processed_path)
    #class別に書き出し
    # box_out.loc[box_out['class'] == 'small'].to_csv(small_path)
    # box_out.loc[box_out['class'] == 'large'].to_csv(large_path)
else:
    print('no car is in this lines')

# 区間長を台数で割って一台の車速を計算する
dis2 = Lperiod / car_num
Speed2 = calSpeed(dis2) * 3.6
print('{}  {}  {:.2f} km/h'.format(key, direc, Speed2))
print('Lperiod is {:.0f} m'.format(Lperiod))
print('num of cars is {:.2f}'.format(car_num))


id 4 RdCL and id 5 RDCL are lines up
id 10 RdCL and id 11 RDCL are lines up
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10]
the most long road lenghts of each group: [108.35851595056445, 100.96994678121706, 175.86649568973277, 135.10505162979572, 183.5385618979173, 59.22376079305299, 186.22494723249068, 298.30820246609983, 121.66975292559033, 137.8659511187845]
each id: [0, 1, 2, 3, 4, 6, 7, 8, 9, 10]
lengths of intersections: [7.446180633532672, 14.97314261668845, 18.559602796242377, 26.325146327549426, 40.56384866655616, 21.98889644785409, 9.432795952241948, 11.699586530693768, 7.66572651094846]

Detection and Paring

    id 0
	inside car: []
	num of cars is 0 in id 0 RdCL

    id 1
	inside car: [55 63]
	num of cars is 2 in id 1 RdCL
	now car: 55 front: 63, rear: None
	now car: 63 front: None, rear: None
	idnext: 63
	idnext: 63
	idnext: None

    id 2
	inside car: [ 8 11 13 17 27 65 66]
	num of cars is 7 in id 2 RdCL
	now car: 8 front: 13, rear: None
	now car: 11 front: 27, rear: 17
	now car: 13 fro

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


	inside car: [16 36 62 69]
	num of cars is 4 in id 3 RdCL
	now car: 16 front: 69, rear: 36
	now car: 36 front: None, rear: 62
	now car: 62 front: None, rear: None
	now car: 69 front: None, rear: None
	idnext: 16
	idnext: 16
	idnext: 36
	idnext: 62
	idnext: None

    id 4
	inside car: [ 71  79  87 113 134 144 149 151 158 161 166]
	num of cars is 11 in id 4 RdCL
	now car: 71 front: 79, rear: 113
	now car: 79 front: 144, rear: None
	now car: 87 front: 134, rear: 151
	now car: 113 front: None, rear: 158
	now car: 134 front: 158, rear: None
	now car: 144 front: None, rear: None
	now car: 149 front: 151, rear: 161
	now car: 151 front: None, rear: None
	now car: 158 front: None, rear: None
	now car: 161 front: None, rear: 166
	now car: 166 front: None, rear: None
	idnext: 79
	idnext: 79
	idnext: 71
	idnext: 113
	idnext: 158
	idnext: 134
	idnext: 87
	idnext: 151
	idnext: 149
	idnext: 161
	idnext: 166
	idnext: None

    id 5
	inside car: [ 85 129 152]
	num of cars is 3 in id 5 RdCL
	now car: 85

In [3]:
key = '1256'
direc = 'in'
day = '20201124'
#bboxのパス
box_path = f'../RodeOutline_HigasiOsaka/bbox/bbox-csv/rd{key}/rd{key}_{day}/rd{key}_{day}-SDrmv.csv'
#車線の読み込みパス
RdCL_path = f'../RodeOutline_HigasiOsaka/RdCL/sb/wgs84/Rd{key}/rd{key}-{direc}.csv'
#車線の処理ピッケルファイル書き出しパス
RdGrp_path = f'../RodeOutline_HigasiOsaka/RdCL/sb/wgs84/Rd{key}/grp-rd{key}-{direc}.pkl'
#bufer内boxの書き出しパス
processed_path = f'../RodeOutline_HigasiOsaka/processed/rd{key}-{direc}-{day}-processed.csv'
#class別のboxの書き出しパス
# small_path = '../RodeOutline_HigasiOsaka/processed/processed-chopped-in-small.csv'
# large_path = '../RodeOutline_HigasiOsaka/processed/processed-chopped-in-large.csv'
#車線を計算するか
cal_Rdlen = True
#車種分け閾値　４．７m + 補正項
class_th = 4.7 + 0.13
th_stop = 5
lineup_th = 4

In [78]:
#計算済みの場合はpickleファイルからRdparを復元
with open(RdGrp_path, "rb") as tf:
    minl_dic = pickle.load(tf)

Rdpar = [i for i in RdCL_df.index.to_list()]
for key in minl_dic.keys():
    for l in minl_dic[key]:
        Rdpar[l[1]] = int(key)
print(Rdpar[:17])
print(len(list(set(Rdpar[:17]))))

[0, 1, 1, 1, 4, 4, 4, 7, 7, 9, 9, 0, 0, 13, 13, 13, 16]
7
